In [2]:
import sys
sys.path.append('../../')
sys.path.append('../')

import fm
from torch.profiler import profile, record_function, ProfilerActivity


In [3]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.fx import subgraph_rewriter, symbolic_trace
import utils
import torch.fx as fx
from torch.fx import Proxy, Graph, GraphModule
from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

replace success!


In [4]:
import time
import torch._dynamo as dynamo


In [5]:
def gen_pattern_replace_and_matcher_for_FM_pow(traced,
                                                  redency_part_slice,unredency_part_slice,
                                                  match_func = None,num_fields = 100, deivce = 'cuda',batch = 4096, dim = 32,prefix = 50
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

  class PatternClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.embed = torch.nn.Embedding(1, 1)

    def forward(self,x):
      x = self.embed(x)
      square_of_sum = torch.sum(x, dim=1) ** 2
      sum_of_square = torch.sum(x ** 2, dim=1)
      ix = square_of_sum - sum_of_square    
      ix = torch.sum(ix, dim=1, keepdim=True)  
      return 0.5 * ix
  def _match(match,ori,pat):
    return True 
  # env  = utils.get_env(traced)
  # target_node = env[target_node_name]
  # target_node_mod = utils.get_target_mod(traced,target_node_name,"_")
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  
  embed_node = node_map[pattern_env['embed']]
  embed_node_module = utils.get_target_mod(traced,embed_node.target)
  

  
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      embedding_config = embed_node_module.weight.data.shape
      self.embed = nn.Embedding(embedding_config[0],embedding_config[1])
      self.embed.weight.data.copy_(embed_node_module.weight.data)
    def forward(self,x):
      embed = self.embed(x)
      square_of_sum = torch.sum(embed,dim=1)** 2
      pow_tensor = torch.empty((batch,num_fields,dim),device='cuda')
      pow_tensor[:,prefix:] = embed[0,prefix:] ** 2
      pow_tensor[:,:prefix] = embed[:,:prefix] ** 2
      sum_of_square = torch.sum(pow_tensor,dim=1)
      ix = square_of_sum - sum_of_square
      ix = torch.sum(ix,dim = 1,keepdim=True)
      return 0.5 * ix
    
  
  return pattern,ReplacementClass(),_match

In [6]:
def workload_FM_pow(num_field, prefix,dim = 64,batch = 4096):
  # print(f"now gen workload of wdl with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}")
  model_ori = fm.FactorizationMachineModel([100 for i in range(num_field)],dim,)
  ori_traced = symbolic_trace(model_ori)
  
  model_modify = fm.FactorizationMachineModel([100 for i in range(num_field)],dim,)
  modify_traced = symbolic_trace(model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_FM_pow(modify_traced,
                                                                      (0,slice(None,prefix,None)),(slice(None,None,None),slice(prefix,None,None)),
                                                                      num_fields=num_field,prefix=prefix,dim= dim,batch=batch)
  matches = subgraph_rewriter.replace_pattern_with_filters(modify_traced, pattern, replace,[match])
  return ori_traced,modify_traced

In [7]:
def calculate_mean_and_variance_manual(data):
    n = len(data)
    mean = sum(data) / n
    variance = sum((x - mean) ** 2 for x in data) / n
    return mean, variance

In [12]:
def gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 32, workload_func = workload_FM_pow,l = [1024,512,256]):
  def run(model):
    model = model.cuda()
    model.eval()
    # traced_model = torch.jit.trace(, torch.randint(low=0, high=88, size=(batch,num_field), dtype=torch.long).cuda())
    compiled_model = torch.compile(model, backend="inductor")
    compiled_model.eval()
    t = torch.randint(low=0, high=88, size=(batch ,num_field), dtype=torch.long).cuda()
    for i in range(6):
        soutput = compiled_model(t)
    torch.cuda.synchronize()
    start_time = time.time()  # 开始计时

    for i in range(100):
        soutput = compiled_model(t)
    # torch.cuda.synchronize()
    end_time = time.time()  # 结束计时
        
        # 计算并打印函数执行所需的时间
    elapsed_time = end_time - start_time
    print(f"time : {elapsed_time * 1000 /100} ms")
  print(f"now gen workload of DFM with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}, batch :{batch}")
  ori, modify = workload_func(num_field,prefix,dim,batch = batch)
  run(ori)
  run(modify)

In [14]:
gen_and_test(num_field = 22 * 5 ,prefix = 10 * 5 , batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 110, prefix: 50, batch :4096


/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/fx/graph.py:1801: UserWarning: Node _tensor_constant0 target _tensor_constant0 _tensor_constant0 of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(
/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/fx/graph.py:1801: UserWarning: Node _tensor_constant0_1 target _tensor_constant0 _tensor_constant0 of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(
/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/fx/graph.py:1801: UserWarning: Node _tensor_constant1 target _tensor_constant1 _tensor_constant1 of  does not reference an nn.Module, nn.Parameter, or buffer, which is what 'get_attr' Nodes typically target
  warnings.warn(


time : 0.18785476684570312 ms
time : 0.2758002281188965 ms
